In [1]:
using Random

# Define sphere function
function sphere_function(x)
    return sum(x .^ 2)
end

# Define Particle struct
mutable struct Particle
    position::Array{Float64}
    velocity::Array{Float64}
    best_position::Array{Float64}
    best_fitness::Float64
    fitness::Float64
end

# Update particle velocity
function update_velocity!(particle::Particle, w, c1, c2, global_best_position)
    r1 = rand(size(particle.position))
    r2 = rand(size(particle.position))
    particle.velocity .= w * particle.velocity .+ c1 .* r1 .* (particle.best_position .- particle.position) .+ c2 .* r2 .* (global_best_position .- particle.position)
end

# Update particle position
function update_position!(particle::Particle, bounds)
    particle.position .= clamp.(particle.position .+ particle.velocity, bounds[1], bounds[2])
end

# Define PSO struct
mutable struct PSO
    objective_function
    num_particles::Int
    dim::Int
    max_iter::Int
    max_vel::Float64
    bounds::Tuple{Float64, Float64}
    w::Float64
    c1::Float64
    c2::Float64
end

# Optimization function
function optimize(pso::PSO)
    swarm = [Particle(rand(pso.bounds[1]:pso.bounds[2], pso.dim), rand(-pso.max_vel:pso.max_vel, pso.dim), zeros(pso.dim), Inf, Inf) for _ in 1:pso.num_particles]
    global_best_position = nothing
    global_best_fitness = Inf

    for _ in 1:pso.max_iter
        for particle in swarm
            particle.fitness = pso.objective_function(particle.position)
            if particle.fitness < particle.best_fitness
                particle.best_fitness = particle.fitness
                copy!(particle.best_position, particle.position)
            end
            if particle.fitness < global_best_fitness
                global_best_fitness = particle.fitness
                global_best_position = copy(particle.position)
            end
        end

        for particle in swarm
            update_velocity!(particle, pso.w, pso.c1, pso.c2, global_best_position)
            update_position!(particle, pso.bounds)
        end
    end

    return global_best_position, global_best_fitness
end

# Define sphere function
sphere_function(x) = sum(x.^2)

# Initialize PSO parameters
pso = PSO(sphere_function, 50, 10, 100, 0.5, (-5.12, 5.12), 0.872, 1.49445, 1.49445)

# Run optimization
best_position, best_fitness = optimize(pso)

# Print results
println("Best position: $best_position")
println("Best fitness: $best_fitness")


Best position: [1.696192, -2.936192, 1.696192, 0.6961919999999999, -1.303808, -1.303808, 0.06380800000000003, 1.0638079999999999, 3.063808, -0.3038080000000002]
Best fitness: 28.874849648639994
